<font color="#CC3D3D"><p>
# 실습-2: 행렬분해 기반 잠재요인 협업 필터링

In [ ]:
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline
from datetime import datetime

from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### 데이터 읽기

In [ ]:
movies = pd.read_csv('/content/drive/Othercomputers/내 노트북/study/10_RecSystem/실습-20230909/movies.csv')
ratings = pd.read_csv('/content/drive/Othercomputers/내 노트북/study/10_RecSystem/실습-20230909/ratings.csv')
print(movies.shape)
print(ratings.shape)

(9724, 3)
(100836, 4)


<font color="blue"><p>
#### 나의 평점 추가하기

In [ ]:
# 나의 사용자 ID
myId = 611

In [ ]:
print("제시되는 영화에 대해 10개까지 평점(1~5점)을 입력하시기 바랍니다.")
print("잘 모르는 영화일 경우 평점을 낮게 주시거나 Enter키를 치시면 됩니다.")

movies_for_rating = pd.read_csv('/content/drive/Othercomputers/내 노트북/study/10_RecSystem/실습-20230909/movies_for_rating.csv')
my_ratings = []
count = 0

df = movies_for_rating.sample(frac=1)
for i in range(df.shape[0]):
    my_rating = input(f"{df.iloc[i].title}: ")
    print(count)
    try:
        score = int(my_rating)
        if score < 1 or score > 5:
            continue
        my_ratings.append((myId, df.iloc[i].movieId, score, int(datetime.now().timestamp())))
        count += 1
    except:
        continue
    if count >= 10:
        break

my_ratings = pd.DataFrame(my_ratings, columns=ratings.columns)
my_ratings.to_csv('my_ratings.csv', index=False)
print("\nMy rating summary:\n", my_ratings.merge(df)[['title', 'genre', 'rating']])

ratings = pd.concat([ratings, my_ratings]).reset_index(drop=True)

제시되는 영화에 대해 10개까지 평점(1~5점)을 입력하시기 바랍니다.
잘 모르는 영화일 경우 평점을 낮게 주시거나 Enter키를 치시면 됩니다.
Mr. Holland's Opus: 1
0
Tombstone: 2
1
Apollo 13: 3
2
The Godfather: 4
3
Space Jam: 5
4
12 Monkeys: 1
5
Independence Day: 2
6
Judgment Night: 3
7
Highlander: 4
8
Schindler's List: 5
9

My rating summary:
                 title      genre  rating
0  Mr. Holland's Opus      Drama       1
1           Tombstone      Drama       2
2           Apollo 13      Drama       3
3       The Godfather      Crime       4
4           Space Jam  Animation       5
5          12 Monkeys     Sci-Fi       1
6    Independence Day     Sci-Fi       2
7      Judgment Night      Crime       3
8          Highlander     Action       4
9    Schindler's List      Drama       5


#### 입력데이터 구성: 평점행렬 만들기

In [ ]:
# title 컬럼을 얻기 이해 movies와 조인 수행
rating_movies = pd.merge(ratings, movies, on='movieId')

# 행은 userID, 열은 title로 pivoting 수행. rating이 없는 값(NaN)은 모두 0으로 변환
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title', fill_value=0)

print(ratings_matrix.shape)
ratings_matrix.head()

(611, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0.0,0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0,0,0.0,0.0,0.0,4.0,0
2,0,0,0.0,0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0
3,0,0,0.0,0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0
4,0,0,0.0,0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0
5,0,0,0.0,0,0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0


#### 행렬 분해: 평점 행렬을 두 개의 잠재요인 행렬(P, Q)로 분해

<img align='left' src='http://drive.google.com/uc?export=view&id=154nu-hKxCt6yc0mWst0mHU944L1-D2vb' width=500>

In [ ]:
def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]

    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다.
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    break_count = 0

    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장.
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

    # SGD기법으로 P와 Q 매트릭스를 계속 업데이트.
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)

    return P, Q

In [ ]:
start_time = time.time()

P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda = 0.01)

print(f"학습시간: {time.time() - start_time}초")

### iteration step :  0  rmse :  2.9548791368448466
### iteration step :  10  rmse :  0.7322243225376687
### iteration step :  20  rmse :  0.5095090349448909
### iteration step :  30  rmse :  0.370108796791834
### iteration step :  40  rmse :  0.29348100227231255
### iteration step :  50  rmse :  0.24946502094654077
### iteration step :  60  rmse :  0.2224201187163551
### iteration step :  70  rmse :  0.20456178300754635
### iteration step :  80  rmse :  0.19200352088080067
### iteration step :  90  rmse :  0.18271889519138604
### iteration step :  100  rmse :  0.17558244684380142
### iteration step :  110  rmse :  0.1699281374288652
### iteration step :  120  rmse :  0.16533871786175797
### iteration step :  130  rmse :  0.16153990466402948
### iteration step :  140  rmse :  0.15834407731258035
### iteration step :  150  rmse :  0.155618597097032
### iteration step :  160  rmse :  0.15326706220850542
### iteration step :  170  rmse :  0.15121773082800224
### iteration step :  180  rms

#### 추천 상품 결정:  개인화된 영화 추천

영화 평점 예측
$$
\Large \hat{R}_{u,j} = P(user factors)_{u} * Q(item factors)^{t}_{j}
$$

In [ ]:
ratings_pred = np.dot(P, Q.T)
ratings_pred = pd.DataFrame(ratings_pred, ratings_matrix.index, ratings_matrix.columns)
ratings_pred

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.237532,3.949917,3.477912,4.616062,4.256744,1.334254,4.767862,2.173030,5.208986,4.279865,...,1.319321,4.012444,3.797460,2.744073,2.678913,4.328053,3.472412,2.079599,4.011733,0.803877
2,3.158821,3.799614,3.364779,4.313768,4.252346,1.096109,3.991197,2.010636,4.016329,3.833486,...,1.081040,4.239841,3.273020,2.665417,2.257579,3.941532,2.082717,1.671694,3.834411,0.795919
3,2.092488,1.880745,1.667188,2.431071,2.350249,0.548221,-0.539346,1.507436,0.922594,1.777931,...,0.813728,2.517707,1.751455,1.813826,1.416401,2.886149,1.347031,0.593588,1.410878,0.500775
4,2.173280,3.071412,2.760102,3.378021,3.173163,0.860805,4.557037,1.654178,3.275662,2.752388,...,1.083956,2.132694,2.930510,1.835082,2.076506,2.009857,1.226190,1.489207,4.070676,0.637778
5,2.273308,3.348187,3.057346,3.708201,3.626140,0.909600,2.494638,1.645964,1.898057,2.743264,...,1.079937,4.257418,3.158263,2.118574,1.877366,4.016062,1.120215,1.746067,2.798974,0.611882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,2.701596,3.496396,3.114957,4.026404,4.128680,1.147217,3.685883,1.880155,4.811396,3.425664,...,0.899185,3.608270,3.300865,2.283239,2.159700,3.842735,2.123324,1.861855,4.911940,0.729425
608,2.152485,3.055794,2.655688,3.803681,3.668513,0.913951,2.084219,1.703268,3.101865,3.356438,...,0.797725,3.098701,2.927521,2.083968,2.033459,4.495179,3.514363,1.852507,2.641817,0.619854
609,2.642618,3.317217,2.914652,3.701082,3.535372,1.018159,2.793434,1.762734,3.515417,3.152431,...,1.017570,3.349141,2.932346,2.173122,2.147491,4.194089,2.609085,1.705313,2.959921,0.647695


Top-N 영화 추천

In [ ]:
# 추천 영화의 수 설정
N = 10
uid = myId

In [ ]:
# id로 지정된 사용자의 모든 영화정보 추출하여 Series로 반환함
# 반환된 user_rating은 영화명(title)을 index로 가지는 Series 객체임
user_rating = ratings_matrix.loc[uid,:]

# user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list로 만듬
already_seen = user_rating[user_rating > 0].index.tolist()

# list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함
unseen_list = [movie for movie in ratings_matrix.columns.tolist() if movie not in already_seen]

# unseen_list에서 가장 평점이 높은 N개의 영화를 추천함
recomm_items = ratings_pred.loc[uid, unseen_list].sort_values(ascending=False)[:N]
list(recomm_items.index)

['Full Monty, The (1997)',
 'Airplane! (1980)',
 'His Girl Friday (1940)',
 'Dead Man Walking (1995)',
 'Emma (1996)',
 'Hoop Dreams (1994)',
 'City of God (Cidade de Deus) (2002)',
 'Casablanca (1942)',
 'Planet Terror (2007)',
 'Dr. No (1962)']

추천성능 평가

In [ ]:
# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 RMSE를 구함
actual, pred = ratings_matrix.values, ratings_pred.values
pred = pred[actual.nonzero()].flatten()
actual = actual[actual.nonzero()].flatten()
rmse = np.sqrt(mean_squared_error(pred, actual))
print(f'RMSE of UBCF@K={N}: ', rmse)

RMSE of UBCF@K=10:  0.1465318497060928


<font color="#CC3D3D"><p>
# End